# Trabalho Branch and Bound 

## Solver Linear

Bibliotecas necessárias para executar o projeto descritas abaixo.

**Dica** Utilize o miniconda para criação de um ambiente virutal python.

```conda create --n <node do env> python = 3.7```
* python = 3.7
* ortools
* numpy
* math
* copy
* random

Por padrão o python vem com todas menos a ortools, podendo ser intalada utilizando o o gerenciador de pacotes pip.

``` pip intall ortools``` 


In [58]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
import numpy as np
import math
import copy

import random

Solver utilizado foi o mesmo do trabalho anteritor com algumas modificações, como a adição de intervalos para as variáveis
e desigualdades lineares para cada restrição.
Por exemplo:

constraint 0:

>> x + y >= 5

constraint 1:

>> x + y < = 10
    
A solução foi o **True para <=** e **False para >=**.

In [59]:
class Solver:
    '''Solver Linear'''
    
    def __init__(self,objective, domain_range = (0,math.inf)):
        '''ORtools instance, as maximize or minimize with domain range [a,b] to variables.'''
        
        self.domain_range = domain_range
        if objective.upper() == 'MAX':
            self.objective = True
        else:
             self.objective = False
    
    def read_problem(self,path):
        '''Read problem the file and return matrix'''
        file  = open(path, "r")
        lines = file.read().split('\n')
        matrix = []

        for i in range(len(lines)):
            line = []
            for j in range(len(lines[i].split())):

                line.append(float(lines[i].split()[j]))
            matrix.append(line)

        file.close()
        
        return matrix
    
    def extrac_propeties1(self,matrix):
        """[This is a function returns number of variables,
        constraints and objective function coefficients.]
        
        Arguments:
            matrix {float} -- [matrix with a problem of PPL]
        
        Returns:
            [(int, int, list)] -- [tuple]
        """    
        
        
        #numbers of variables
        num_vars = int(matrix[0][0])
        
        #numbers of constraints
        num_constraints = int(matrix[0][1])

        #objective function coefficients
        fo_coefficients = []

        for i in range(len(matrix[1])):
            aux = matrix[1][i]
            fo_coefficients.append(aux)
        
        return num_vars, num_constraints, fo_coefficients
    
    def extrac_propeties(self,file):
        '''As extrac_propeties1, but input is a file.'''
        matrix = self.read_problem(file)
        return self.extrac_propeties1(matrix)
        
    
    def resolve_of_the_problem(self,num_vars, num_constraints, fo_coefficients, matrix):
        '''Resolve_of_the_problem and return number of variables, contraints and FO coefficients'''
        
        #create solver of PPL
        self.solver = pywraplp.Solver('LinearProgrammingTest',
                            pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
        
        #var with infinity value
        inf = self.solver.infinity()
        #list to vars of the problem
        vars = []
        '''INTERVALOS DAS VARIÁVEIS'''
        a,b = self.domain_range
        #for all x>=0 and x<=1
        for i in range(num_vars):
            name = 'x'+str(i+1)
            vars.append(self.solver.NumVar(a,b,name))    
        
        #constraints
        constraints = []
        for i in range(2,len(matrix)):
            aux = len(matrix[i]) - 2
            
            #menor igual
            if matrix[i][aux+1]:
                constraints.append(self.solver.Constraint(0,matrix[i][aux],f'constraint{i}')) 
            #maior igual                   
            else:
                constraints.append(self.solver.Constraint(matrix[i][aux],inf,f'constraint{i}'))
                
            for j in range(aux):
                constraints[i-2].SetCoefficient(vars[j],matrix[i][j])
        
        #FO (objective function)
        objective = self.solver.Objective()
        #coefficients da FO
        FO = matrix[1]

        for i in range(len(FO)):
            objective.SetCoefficient(vars[i],FO[i])

        if self.objective:
             #SetMaximization
            objective.SetMaximization()
        else:
            #SetMinimization
            objective.SetMinimization()

        #solve of the problem
        status  = self.solver.Solve()
        return vars, status, self.solver
           
    def convert_for_dual_form(self,matrix):
        '''Convert matrix for dual problem'''
        
        line = matrix[0]
        line.reverse()
        fo = matrix[1]

        matrix = matrix[2:]
        
        new_matrix = [line]
        
        n =  len(matrix[0])-1
        for i in range(n):
            l = []
            for j in range(len(matrix)):
                l.append(matrix[j][i])
                
            l.append(fo[i])
            new_matrix.append(l)
        
        new_fo = [matrix[i][n] for i in range(len(matrix))]
        new_matrix.insert(1,new_fo)
        
        return new_matrix
        
    def print_solution(self, vars,status, solver_solution, fo_coefficients):
        '''Print solution'''
            
        opt_solution = 0
        print('Number of variables =', solver_solution.NumVariables())
        print('Number of constraints =', solver_solution.NumConstraints())
        
        if status == solver_solution.OPTIMAL:    
            print('Solution:')
            
            for i in range(len(vars)):
                print(str(vars[i])+' = {0}'.format(vars[i].solution_value()))
                
            for i in range(len(vars)):
                opt_solution += fo_coefficients[i] * vars[i].solution_value()
            
            # The objective value of the solution.
            print('Optimal objective value =', opt_solution)
        else:
            if solver_solution.FEASIBLE:
                print('A potentially suboptimal solution was found.')
            else:
                print('The solver could not solve the problem.')
    
    
    def convert_form(self,matrix):
        '''Make all matrix constraints as greater than equal <=,
            where True is less than equal and False is greater than equal'''
        if self.objective:
            size = len(matrix[2])
            for i in range(2,len(matrix)):
                matrix[i].append(True)
            return matrix
        else:
            size = len(matrix[2])
            for i in range(2,len(matrix)):
                matrix[i].append(False)
            return matrix

    def solve(self,file):
        '''Solve the problem and print at screen'''
        #extraindo número de variáveis, restrições e coeficientes da função objetivo
        num_vars, num_constraints , fo_coefficients = self.extrac_propeties(path)
        
        matrix = self.read_problem(file)
        matrix = self.convert_form(matrix)
        print(matrix)
        #resolução do problema primal
        vars, status, solver_solution = self.resolve_of_the_problem(num_vars,num_constraints,fo_coefficients,matrix)

        self.print_solution(vars, status, solver_solution, fo_coefficients)


### Main for solver linear

In [60]:
path = "problema_01.txt"
print('\nProblema primal')

#MIN ou MAX
primal_solver = Solver('min', domain_range = (0,1))
primal_solver.solve(path)


Problema primal
[[3.0, 2.0], [5.0, 10.0, 8.0], [3.0, 5.0, 2.0, 6.0, False], [4.0, 4.0, 4.0, 7.0, False]]
Number of variables = 3
Number of constraints = 2
Solution:
x1 = 1.0
x2 = 0.5
x3 = 0.25000000000000017
Optimal objective value = 12.000000000000002


# Branch and Bound

Na classe **Tree** está a implementação do método **branch and bound**, para  problemas de Minimização, para variáveis \begin{equation*}x \in {0,1} \end{equation*}

Os métodos **BSF** e **DSF** servem para montar a árvore e depois percorrer em profundidade e pegar a melhor solução, respectivamente.


In [61]:
class Node():
    '''Node class contains data of problem branch and bound.'''
    count = 0
    def __init__(self, problem, solution, left = None, right = None, value = 0):
        self.problem = problem
        self.solution = solution
        self.value = value
        
        self.left = left
        self.right = right   
        Node.count += 1
        self.id = self.count

##

In [62]:

class Tree():
    '''Branch and bound tree.'''
    
    def __init__(self, problem):
        '''Define solver and limits.'''
        self.solver = Solver('min',domain_range = (0,1))
        self.problem = self.solver.read_problem(problem)
        
        self.high = math.inf
        self.low = math.inf
        self.root = None
        
    def check_integer(self,node):
        '''Check integrity'''
        #verifica se há alguma variável não inteira
        vars = node.solution[0]
        for i in range(len(vars)):
            a = vars[i].solution_value()    
            if a != int(a):
                return False
        return True
    
    def check_limit(self,node):
        '''Checks and updates primal and dual limits'''
        
        if self.check_integer(node):#verifica integralidade
            if node.value < self.low:
                self.low = node.value
                return True
            
            else:#parar por limitante
                return False
        else:
            #if node.value > self.low and node.value < self.high:
                #self.low = node.value
            return True
            
    def check(self, node):
        '''Check intgrity and limits'''
        
        status = node.solution[1]
        solver_solution = node.solution[2]
        #verifica se é viável
        if status == solver_solution.OPTIMAL:
              
            if self.check_limit(node):
                return True
            return False
        
        else:
            return False
    
    def insert_constraint(self, problem, solution):
        '''Create 2 new problems with new constraints'''        
        vars, status, solver_solution = solution
        
        select = [math.inf for i in range(len(vars))]
        
        for i,var in enumerate(vars):
            a = var.solution_value()
            
            if(a!=int(a)):
                aux = abs((a%1)-0.5)
                select[i] = round(aux,3)
        #indice da variável float com fração mais proximo de 0.5
        j = select.index(min(select))
        
        size_constratin = len(vars) + 2
        problem[0][1] = problem[0][1] + 1
        
        new_constraint1 = [0] *size_constratin
        new_constraint2 = [0] *size_constratin
        #coeficiente da variável na restrição
        new_constraint1[j] = 1 
        #primera restrição <=
        new_constraint1[-2] = int(vars[j].solution_value())
        new_constraint1[-1] = True
        
        new_problem1 = copy.deepcopy(problem)
        new_problem1.append(new_constraint1)
        
        #coeficiente da variável na restrição
        new_constraint2[j] = 1 
        #segunda restrição >=
        new_constraint2[-2] = int(vars[j].solution_value())+1
        new_constraint2[-1] = False
        
        new_problem2 = copy.deepcopy(problem)
        new_problem2.append(new_constraint2)
        
        
        return new_problem1, new_problem2
    
    def solver_problem(self,problem):
        '''Solve problem for branch and bound'''
        
        num_vars, num_constraints, self.fo_coefficients = self.solver.extrac_propeties1(problem)
        
        # resolução do problema primal
        vars, status, solver_solution = self.solver.resolve_of_the_problem(num_vars,
                                                                             num_constraints,
                                                                             self.fo_coefficients,
                                                                             problem)
        
        
        return vars, status, solver_solution
        
    def print_node(self,node):
        '''Print values of variables solution'''
       
        print('NODE VALUE:',node.value,'\n')
        vars, status, solver_solution = node.solution
        opt = 0
        for i in range(len(vars)):
            print(str(vars[i])+' = {0}'.format(vars[i].solution_value()))
            opt += vars[i].solution_value()
        print('****************************************')
    
    def dsf(self, node):
        '''Depth-First-Search (DFS)'''
        if node is not None:
            self.print_node(node)
            self.dsf(node.left)
            self.dsf(node.right)
        if node is not None and self.check_integer(node) and node.value == self.low:
            self.solve = node
            
            
    def DSF(self):
        '''Depth-First-Search (DFS) start root.'''
        self.dsf(self.root)

    def create_node(self,problem,solution):
        '''Create node with current solution value.'''
        vars = solution[0]
        opt = 0
        for i in range(len(vars)):
            opt += self.fo_coefficients[i] * vars[i].solution_value()
        return Node(problem,solution, value = opt)
                

    def bsf(self,node): 
        '''Breadth first search'''
        if node is not None:
            self.deque = [node]
            
            while(len(self.deque)):
                aux_node = self.deque.pop(0)
                self.print_node(aux_node)
                print('limite high',self.high)
                print('limite low',self.low)
                print('')
                
                # cria novos 2 problemas com novas restrições 
                new_prob1, new_prob2 = self.insert_constraint(aux_node.problem, aux_node.solution)
                
                # criar um novo nó para esquerda
                vars, status, solver_solution = self.solver_problem(new_prob1)
                solution = (vars, status, solver_solution)

                left_node = self.create_node(new_prob1,solution)
                
                isContinue = self.check(left_node)
                if isContinue:
                    aux_node.left = left_node
                    self.deque.append(aux_node.left)
                else:
                    #print('\nnão foi para esquerda\n')
                    pass

                #criar um novo nó para direita
                vars, status, solver_solution = self.solver_problem(new_prob2)
                solution = (vars, status, solver_solution)
                
                right_node = self.create_node(new_prob2,solution)
        
                isContinue = self.check(right_node)
                if isContinue:
                    aux_node.right = right_node
                    self.deque.append(aux_node.right)
                else:
                    #print('não foi para direita')
                    pass
                
                
    def BSF(self, node = None):
        '''Breadth first search start root'''
        if node is None:
            Node.count = 0
            self.problem = self.solver.convert_form(self.problem)
            
            vars, status, solver_solution = self.solver_problem(self.problem )
            solution = (vars, status, solver_solution)
            problem = copy.deepcopy(self.problem)
            
            node = self.create_node(problem,solution)
            self.root = node
            #self.low = self.root.value
            
            # verifica se a solução é viável ou inteira
            isContinue = self.check(self.root)
            if isContinue:
                self.bsf(self.root)      
        
        
        

### Test

In [63]:
#Lendo problema
path = "problema_01.txt"

In [64]:
my_tree = Tree(path)
my_tree.BSF()

NODE VALUE: 12.000000000000002 

x1 = 1.0
x2 = 0.5
x3 = 0.25000000000000017
****************************************
limite high inf
limite low inf

NODE VALUE: 13.75 

x1 = 0.75
x2 = 1.0
x3 = 0.0
****************************************
limite high inf
limite low inf

NODE VALUE: 16.0 

x1 = 0.0
x2 = 1.0
x3 = 0.75
****************************************
limite high inf
limite low 15.0

NODE VALUE: 15.0 

x1 = 1.0
x2 = 1.0
x3 = 0.0
****************************************
limite high inf
limite low 15.0



In [65]:
# Limite
my_tree.low

15.0

In [66]:
# Percorre a árvore por profundidade e pega melhor solução 
my_tree.DSF()

NODE VALUE: 12.000000000000002 

x1 = 1.0
x2 = 0.5
x3 = 0.25000000000000017
****************************************
NODE VALUE: 13.75 

x1 = 0.75
x2 = 1.0
x3 = 0.0
****************************************
NODE VALUE: 16.0 

x1 = 0.0
x2 = 1.0
x3 = 0.75
****************************************
NODE VALUE: 15.0 

x1 = 1.0
x2 = 1.0
x3 = 0.0
****************************************


In [67]:
# Solução
try:
    my_tree.print_node(my_tree.solve)
except Exception as e:
    print('Not have an optimal solution!')

NODE VALUE: 15.0 

x1 = 1.0
x2 = 1.0
x3 = 0.0
****************************************


### Teste com ortools mixed programing

In [68]:
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver('simple_mip_program',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

infinity = solver.infinity()
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, 1, 'x')
y = solver.IntVar(0.0, 1, 'y')
z = solver.IntVar(0.0, 1, 'z')

print('Number of variables =', solver.NumVariables())

solver.Add(3*x + 5*y + 2*z >= 6)
solver.Add(4*x + 4*y + 4*z >= 7)

print('Number of constraints =', solver.NumConstraints())

solver.Minimize(5*x + 10 * y + 8*z)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
    print('z =', z.solution_value())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 3
Number of constraints = 2
Solution:
Objective value = 15.0
x = 1.0
y = 1.0
z = 0.0
